In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,493 kB]
Get:14 http://security

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-16 05:05:37--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.26MB/s    in 0.7s    

2022-11-16 05:05:39 (1.26 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

dfShoes = spark.read.option("header", "true").csv("file://" + SparkFiles.get("amazon_reviews_us_Beauty_v1_00.tsv.gz"), sep='\t', inferSchema=True)
dfShoes.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+----------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|marketplace|customer_id|review_id     |product_id|product_parent|product_title                                                                                 |product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline              

In [5]:
dropna_df = dfShoes.dropna(how='any')
dropna_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

In [10]:
dfShoesCleaned = dropna_df.dropDuplicates(['product_id'])
dfShoesCleaned.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   13986795|R2FDU5M81HJUEL|0451192362|      52230657|how stella got he...|          Beauty|          2|            1|          1|   N|                Y|Should have looke...|I did not like th...| 2013-07-09|
|         US|   43882812|R34FARMWJEFAKM|0702001996|     307179209|  Mastiha Hand Cream|          Beauty|          5|    

In [11]:
rows = dfShoesCleaned.count()
rows

588771

In [12]:
dfShoesCleaned = dfShoesCleaned.dropDuplicates(['customer_id'])
dfShoesCleaned.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|      10053|R20JWFTP4MYWAQ|B003L2CPW0|     959460701|Champs Cuba Gold ...|          Beauty|          5|            1|          1|   N|                Y|Oh does he smell ...|Love it. No compl...| 2015-05-17|
|         US|      10104| RYXFP391C0VLY|B006EN98DC|     633927395|Color Charm Perma...|          Beauty|          5|    

In [13]:
rows2 = dfShoesCleaned.count()
rows2

466687

In [14]:
review_id_table = dfShoesCleaned.select(["review_id","customer_id","product_id","product_parent","review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R20JWFTP4MYWAQ|      10053|B003L2CPW0|     959460701| 2015-05-17|
|R3JR51ORU0B3UV|      10117|B00IP99C2U|     730053931| 2014-09-29|
| R7YBEPXRM9A5I|      10206|B00F90Y21O|     623397371| 2014-03-12|
|R38WUVXO558JZG|      10236|B003842346|     698979028| 2015-07-19|
| RH5UQ2T57H9E6|      10272|B000VOJLIC|     236485222| 2015-07-22|
| R9HCFAE9GX5G3|      10293|B00NM7NU6E|     184417519| 2014-12-21|
|R32Z15CVSFMDK0|      10365|B00H1DVEAY|     657964262| 2015-07-28|
| RFS1F128L94K0|      10485|B006NBBOIC|      59130842| 2014-02-28|
|R1D0D2UAKZXS43|      10498|B003KJY964|      68318408| 2012-11-25|
| RAB30J58FJCON|      10517|B004YDWNHC|     416308773| 2013-06-07|
|R13ZZKAI7TDXJZ|      10776|B00DUWN7XW|     721249061| 2015-06-08|
| RIEB47POL0NBS|      10807|B00M0J1F62|     775948398| 2015-08

In [15]:
products = dfShoesCleaned.select(["product_id","product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B003L2CPW0|Champs Cuba Gold ...|
|B00IP99C2U|Mary Kay Forever ...|
|B00F90Y21O|Apple Iphone 4, 4...|
|B003842346|EZ Flow Pastel Co...|
|B000VOJLIC|Baldessarini Ambr...|
|B00NM7NU6E|Paixpays Cosplay ...|
|B00H1DVEAY|White Bobby Pins ...|
|B006NBBOIC|Freetress Equal L...|
|B003KJY964|Paul Mitchell Tea...|
|B004YDWNHC|Luscious Kisses B...|
|B00DUWN7XW|Revlon Colorstay ...|
|B00M0J1F62|The Veda Company ...|
|B00K8U9QI4|DGI MART Party Su...|
|B00K7VAIFO|CoverGirl Queen C...|
|B00OQAXOYE|SHANY Paraben Fre...|
|B005ZUH1VG|Michel Design Wor...|
|B00GYAXLAG|Mary Kay Cologne ...|
|B00JIYAGMG|The Blades Grim G...|
|B00DUTZDQO|TUBE 18pcs Profes...|
|B006HILLIO|Karen Low Pure Bl...|
+----------+--------------------+
only showing top 20 rows



In [16]:
from pyspark.sql.functions import monotonically_increasing_id 

dfShoesCleaned = dfShoesCleaned.select("*").withColumn("customer_count", monotonically_increasing_id())
dfShoesCleaned.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|customer_count|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------+
|         US|      10053|R20JWFTP4MYWAQ|B003L2CPW0|     959460701|Champs Cuba Gold ...|          Beauty|          5|            1|          1|   N|                Y|Oh does he smell ...|Love it. No compl...| 2015-05-17|             0|
|         US|      10104| RYXFP391C0VLY|B006EN98DC|     6339

In [17]:
customers = dfShoesCleaned.select(["customer_id","customer_count"])
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|     455082|             0|
|   23797097|             1|
|   46012211|             2|
|   33869939|             3|
|   11227090|             4|
|    2895607|             5|
|   35253137|             6|
|   15206338|             7|
|   29071735|             8|
|   50032896|             9|
|   10132158|            10|
|   21721563|            11|
|   32879728|            12|
|   45170959|            13|
|   44371228|            14|
|   43615942|            15|
|   43340917|            16|
|   29348097|            17|
|   15338383|            18|
|   25923342|            19|
+-----------+--------------+
only showing top 20 rows



In [18]:
vine_table = dfShoesCleaned.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R20JWFTP4MYWAQ|          5|            1|          1|   N|
|R3JR51ORU0B3UV|          5|            1|          1|   N|
| R7YBEPXRM9A5I|          4|            0|          0|   N|
|R38WUVXO558JZG|          2|            1|          1|   N|
| RH5UQ2T57H9E6|          5|            0|          0|   N|
| R9HCFAE9GX5G3|          5|            0|          0|   N|
|R32Z15CVSFMDK0|          3|            1|          1|   N|
| RFS1F128L94K0|          5|            2|          2|   N|
|R1D0D2UAKZXS43|          2|            0|          0|   N|
| RAB30J58FJCON|          5|            0|          0|   N|
|R13ZZKAI7TDXJZ|          5|            0|          0|   N|
| RIEB47POL0NBS|          5|            0|          0|   N|
|R204ZHFJ8JMWLU|          2|            0|          0|   N|
|R1KGXDZPVVHNUE|          3|            

In [19]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}


In [21]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [22]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [23]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [24]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)